## Cabeçalho
### Objetivo

Funções para a extração dos dados das APIs de dados abertos da Câmara dos Deputados.

Após extração, cada função será acionada nos respectivos notebooks.

### Sobre
**Data de crição:** 26/09/2025

**Responsável:** Wandrieli Nery Barbosa

In [0]:
import requests
import time
from datetime import datetime
from pyspark.sql import functions as F
import re
import shutil
import os

Requisição segura

In [0]:
#Função segura para extração com retry e tratamento de erro 
def safe_get(url, params, tentativas=3, espera=5, timeout=30):
    
    """
    Busca segura necessária para novas requisições e tratamento de erros (erros de timeout)
    """

    for i in range(tentativas):
        try:
            response = requests.get(
                url, params=params, headers={"accept": "application/json"}, timeout=timeout
            )
            response.raise_for_status()
            return response
        except requests.exceptions.HTTPError as e:
            print(f"[ERRO HTTP] Tenativa {i+1}/{tentativas} - {e}")
        except requests.exceptions.RequestException as e:
            print(f"[ERRO REDE] Tenativa {i+1}/{tentativas} - {e}")
        time.sleep(espera)
    print(f"Falha ao acessar {url} com params={params}")
    return None

Funções nb_deputados

In [0]:
URL_DEPUTADOS = "https://dadosabertos.camara.leg.br/api/v2/deputados"

def get_deputados(pagina=1, itens=100):
   
    """
    Requisição para API
    pagina: número da página
    itens: número de itens por página (máx 100)
    Retorna a lista de deputados
    """

    url = f"{URL_DEPUTADOS}?pagina={pagina}&itens={itens}"
    response = requests.get(url, headers={"accept": "application/json"})
    response.raise_for_status()
    return response.json()["dados"]


def get_all_deputados(itens_por_pagina=100, limite_paginas=None):
    
    """
    Busca todos os deputados da API (com paginação).
    itens_por_pagina: máximo de registros por página (default 100)
    limite_paginas: Quais páginas, onde None = todas disponíveis
    lista todos os deputados
    """

    deputados = []
    pagina = 1
    while True:
        dados = get_deputados(pagina=pagina, itens=itens_por_pagina)
        if not dados:  
            break
        deputados.extend(dados)
        pagina += 1
        if limite_paginas and pagina > limite_paginas:
            break
    return deputados


def deputados_to_dataframe(deputados):
    
    """
    Converte lista de deputados para DataFrame Spark.
    """

    df =  spark.createDataFrame(deputados) 

    df = df.withColumn('data_ingestao', F.lit(datetime.now())) #adicionando as colunas de metadados
    df = df.withColumn("url_origem", F.lit(URL_DEPUTADOS)) #adicionando as colunas de metadados

    return df


Funções nb_proposicoes

In [0]:
URL_PROPOSICOES = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"



def get_proposicoes(pagina=1, itens =100, ano=None, siglaTipo=None):
    
    """
    Busca a lista de proposições da API.
    pagina: número da página (Obrigatório)
    itens: número de itens por página (máx 100) (Obrigatório)
    ano: ano da proposição (opcional)
    siglaTipo: sigla do tipo da proposição (opcional)
    return: retorna a lista de proposições
    """

    params = {k: v for k, v in {
        "pagina": pagina,
        "itens": itens,
        "ano": ano,
        "siglaTipo": siglaTipo
    }.items() if v is not None
    }
    response = requests.get(URL_PROPOSICOES, params=params)
    if not response:
        return []

    return response.json()["dados"]

def get_all_proposicoes(itens_por_pagina=100, limite_paginas=None, ano=None, siglaTipo=None):
    
    """
    Busca a lista de proposições, considerando a paginação.
    itens_por_pagina: máximo de registros por página (default 100)
    limite_paginas: Quais páginas, onde None = todas disponíveis
    ano: ano da proposição (opcional)
    siglaTipo: sigla do tipo da proposição (opcional)
    return: lista de proposições
    """

    proposicoes = []
    pagina = 1
    while True:
        dados = get_proposicoes(pagina=pagina, itens=itens_por_pagina, ano=ano, siglaTipo=siglaTipo)
        if not dados:
            break
        proposicoes.extend(dados)
        print(f"Página {pagina} carregada ({len(dados)} registros)")
        pagina += 1
        if limite_paginas and pagina > limite_paginas:
            break
    return proposicoes
    
def proposicoes_to_dataframe(proposicoes):
    
    """
    Converte lista de proposições para DataFrame Spark.
    """

    df = spark.createDataFrame(proposicoes)

    df = df.withColumn('data_ingestao', F.lit(datetime.now())) #adicionando as colunas de metadados
    df = df.withColumn("url_origem", F.lit(URL_PROPOSICOES)) #adicionando as colunas de metadados

    return df

Funções nb_votacoes

In [0]:
URL_VOTACOES = "https://dadosabertos.camara.leg.br/api/v2/votacoes"


def get_votacoes(pagina=1, itens=100,dataInicio=None, dataFim=None):
    """
    Busca votações por página.
    """

    params = {k: v for k, v in{
        "pagina": pagina,
        "itens": itens,
        "dataInicio": dataInicio,
        "dataFim": dataFim
    }.items() if v is not None}

    response = safe_get(URL_VOTACOES, params=params)
    if not response:
        return []

    return response.json()["dados"]

def get_all_votacoes(itens_pagina=100, limite_pagina=None, dataInicio=None, dataFim=None):

    """
    Busca todas as votações da Câmara com paginação
    """

    votacoes = []
    pagina = 1
    while True:
        dados = get_votacoes(pagina=pagina, itens=itens_pagina, dataInicio=dataInicio, dataFim=dataFim)
        if not dados:
            break
        votacoes.extend(dados)
        print(f"Página {pagina} carregada ({len(dados)} registros)")
        pagina +=1
        if limite_pagina and pagina > limite_pagina:
            break
    return votacoes
    
def votacoes_to_dataframe(votacoes):

    """
    Converte lista de votações para DataFrame Spark.
    """

    df = spark.createDataFrame(votacoes)

    df = df.withColumn('data_ingestao', F.lit(datetime.now())) #adicionando as colunas de metadados
    df = df.withColumn("url_origem", F.lit(URL_VOTACOES)) #adicionando as colunas de metadados

    return df

Funções nb_eventos

In [0]:
URL_EVENTOS = "https://dadosabertos.camara.leg.br/api/v2/eventos"


def get_eventos(pagina=1, itens=100, dataInicio=None, dataFim=None):

    """
    Busca eventos da API da Câmara.
    pagina: número da página (obrigatório)
    itens: nº máximo de registros por página (máx=100)
    dataInicio: filtrar a partir desta data (yyyy-mm-dd)
    dataFim: filtrar até esta data (yyyy-mm-dd)
    """

    params = {
        "pagina": pagina,
        "itens": itens,
        "dataInicio": dataInicio,
        "dataFim": dataFim
    }
    params = {k: v for k, v in params.items() if v is not None}

    response = safe_get(URL_EVENTOS, params=params)
    if not response:
        return []
    return response.json()["dados"]


def get_all_eventos(itens_por_pagina=100, limite_paginas=None, dataInicio=None, dataFim=None):

    """
    Lista todos os eventos (com paginação)
    """

    eventos = []
    pagina = 1
    while True:
        dados = get_eventos(pagina=pagina, itens=itens_por_pagina,
                            dataInicio=dataInicio, dataFim=dataFim)
        if not dados:
            break
        eventos.extend(dados)
        print(f"Página {pagina} carregada ({len(dados)} registros)")
        pagina += 1
        if limite_paginas and pagina > limite_paginas:
            break
    return eventos



def eventos_to_dataframe(eventos):
    
    """
    Converte lista de eventos para DataFrame Spark.
    """

    df = spark.createDataFrame(eventos)
    df = df.withColumn("data_ingestao", F.lit(datetime.now())) #adicionando as colunas de metadados
    df = df.withColumn("url_origem", F.lit(URL_EVENTOS)) #adicionando as colunas de metadados
    return df

Funções nb_discursos

Função - Salva Dados

In [0]:
def save_data(df, tabela="dados_bronze"):
    
    """
    Salva o DataFrame como tabela no Databricks.
    tabela: Nome da tabela a ser criada
    """
    
    spark.sql(f'DROP TABLE IF EXISTS {tabela}')
    df.write.saveAsTable(tabela)
    print(f"Tabela '{tabela}' criada/atualizada com sucesso!") #validações dos parâmetros